In [1]:
import minsearch
import json 

In [2]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents = []

for course_dict in docs_raw:
    # course: de zoomcamp -> q/a list 
    # Noramlize the course dict as list of doc
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
index = minsearch.Index(
    text_fields=['question', 'text', 'section'],
    keyword_fields=['course']
)

In [5]:
q = 'the course has already started, can I still enroll?'
index.fit(documents)

boost = {'question': 3, 'section': 0.5, 'text': 1}

results = index.search(
    query=q,
    filter_dict={"course": "data-engineering-zoomcamp"},
    boost_dict=boost,
    num_results=5
)

In [6]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [26]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, ouput NONE 

QUESTION: {question}

CONTEXT: {context}
"""

context = ''
for doc in results:
    context = context + f"section: {doc['section']}\nquestion:  {doc['question']}\ntext:  {doc['text']}\n\n"

prompt = prompt_template.format(question=q, context=context).strip()

In [1]:
!poetry add openai

Using version ^1.37.1 for openai

Updating dependencies
Resolving dependencies... (2.1s)Resolving dependencies... (1.4s)Resolving dependencies... (1.5s)

Package operations: 2 installs, 1 update, 0 removals

  - Updating numpy (1.26.4 -> 2.0.0): Pending...
  - Updating numpy (1.26.4 -> 2.0.0): Installing...
  - Updating numpy (1.26.4 -> 2.0.0)
  - Installing distro (1.9.0): Pending...
  - Installing distro (1.9.0): Downloading... 0%
  - Installing distro (1.9.0): Downloading... 100%
  - Installing distro (1.9.0): Installing...
  - Installing distro (1.9.0)
  - Installing openai (1.37.1): Pending...
  - Installing openai (1.37.1): Downloading... 0%
  - Installing openai (1.37.1): Downloading... 100%
  - Installing openai (1.37.1): Installing...
  - Installing openai (1.37.1)

Writing lock file


In [5]:
from openai import OpenAI
client = OpenAI(
    base_url='https://pd-20tik-llm-zoomca-fd9cd5a254e64e34b0bc1f706b653626.community.saturnenterprise.io/v1',
    api_key='ollama',
    default_headers = {"Authorization": f"token {user_token}"}
)

In [7]:
llm('Tell me how to write python code')

'Sure, I\'d be happy to guide you through writing your first Python code. Here\'s a step-by-step guide:\n\n1. **Environment Setup:**\n   Before you start coding, you need to have Python installed on your computer. You can download Python from the official website: https://www.python.org/downloads/\n\n   Once downloaded, install it and ensure that Python is added to your system\'s PATH. You should be able to open a command prompt or terminal window and type `python` (or `python3`) without any errors.\n\n2. **Your First Code:**\n   Let\'s start with the classic "Hello, World!" program.\n\n   Open a text editor like Notepad (on Windows) or TextEdit (on Mac), and write the following lines:\n\n   ```python\n   print("Hello, World!")\n   ```\n\n   Save this file as `hello_world.py`. The `.py` extension is used for Python files.\n\n3. **Running Your Code:**\n   - On Windows: Open Command Prompt, navigate to the folder where you saved your file using the `cd` command (e.g., `cd C:\\path\\to\\y

In [8]:
print(_)

Sure, I'd be happy to guide you through writing your first Python code. Here's a step-by-step guide:

1. **Environment Setup:**
   Before you start coding, you need to have Python installed on your computer. You can download Python from the official website: https://www.python.org/downloads/

   Once downloaded, install it and ensure that Python is added to your system's PATH. You should be able to open a command prompt or terminal window and type `python` (or `python3`) without any errors.

2. **Your First Code:**
   Let's start with the classic "Hello, World!" program.

   Open a text editor like Notepad (on Windows) or TextEdit (on Mac), and write the following lines:

   ```python
   print("Hello, World!")
   ```

   Save this file as `hello_world.py`. The `.py` extension is used for Python files.

3. **Running Your Code:**
   - On Windows: Open Command Prompt, navigate to the folder where you saved your file using the `cd` command (e.g., `cd C:\path\to\your\folder`), and then run 

In [43]:
def search(query): 
    boost = {'question': 3, 'section': 0.5, 'text': 1}

    results = index.search(
        query=query,
        filter_dict={"course": "data-engineering-zoomcamp"},
        boost_dict=boost,
        num_results=5
    )
    return results

In [36]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, ouput NONE 

QUESTION: {question}

CONTEXT: {context}
"""

    context = ''
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion:  {doc['question']}\ntext:  {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt 

In [3]:
def llm(prompt):
    response = client.chat.completions.create(
        model='mistral-nemo',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [47]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer 

In [48]:
rag('The course has already started, can I still enroll?')

'Yes, even if the course has already started, you can still enroll and submit the homeworks. However, be aware of the deadlines for turning in the final projects. The course materials will be kept after the course finishes, so you can continue following the course at your own pace.'

In [53]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 
# es_client.info()
index_setting = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions'

es_client.indices.create(index=index_name,body=index_setting)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [56]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:23<00:00, 39.83it/s]


In [57]:
query = 'The course has already started, can I still enroll?'

In [69]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [72]:
elastic_search('What if I don\'t have google account')

[{'text': "Problem: The following error occurs when attempting to export data from Mage to a GCS bucket. Assigned service account doesn’t have the necessary permissions access Google Cloud Storage Bucket\nPermissionError: [Errno 13] google::cloud::Status(PERMISSION_DENIED: Permanent error GetBucketMetadata:... .iam.gserviceaccount.com does not have storage.buckets.get access to the Google Cloud Storage bucket. Permission 'storage.buckets.get' denied on resource (or it may not exist). error_info={reason=forbidden, domain=global, metadata={http_status_code=403}}). Detail: [errno 13] Permission denied\nSolution: Add Cloud Storage Admin role to the service account:\nGo to project in Google Cloud Console>IAM & Admin>IAM\nClick Edit principal (pencil symbol) to the right of the service account you are using\nClick + ADD ANOTHER ROLE\nSelect Cloud Storage>Storage Admin\nClick Save",
  'section': 'Module 2: Workflow Orchestration',
  'question': 'Mage - PermissionError service account does not

In [73]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer 

In [74]:
rag('The course has already started, can I still enroll?')

"Yes, even if the course has already started, you can still enroll and submit homeworks. However, be aware of the deadlines for turning in the final projects and don't leave everything for the last minute. You can also start by installing and setting up all the dependencies and requirements before beginning the course.\n\nReference(s):\nsection: General course-related questions\nquestion: Course - Can I still join the course after the start date?\ntext: Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.\n\nsection: General course-related questions\nquestion: Course - What can I do before the course starts?\ntext: You can start by installing and setting up all the dependencies and requirements: Google cloud account Google Cloud SDK Python 3 (installed with Anaconda) Terraform Git Look over the prerequisites and syllabus to see if 